In [ ]:
import glob
import os
import numpy as np
from IPython.display import HTML

import sys
# sys.path.append("/home/shoinoue/rsc/projects/sho-meta/pyfiles/")
sys.path.append("/Users/shoahoshoaho/Git/sho-meta/pyfiles/")
from json_ext import load_json
def text_normalization_dialogtts(text):
    text = text.replace("[laughter]", "")
    replacements = {
        "%": "percent",
        "&": "and",
        "$": "dollar",
        "@": "at",
    }
    for symbol, word in replacements.items():
        text = text.replace(symbol, f" {word} ")
    return text

base_dir = "./audiosamples/"
emo2color = {
    "Angry": "red",
    "Happy": "orange",
    "Neutral": "green",
    "Sad": "blue",
}
target_emotions = ["angry", "happy", "sad"]

In [ ]:
              # <p>
              #   <sup>1</sup>FAIR at Meta<br>
              #   <sup>*</sup>Work done at Meta<br>
              # </p>

In [ ]:
webtitle = "EmoDialogTTS Demo Page"
title = "EmoDialogTTS: Generating Empathetic Speech Dialogs with Turn-Level Emotion Conditioning"
abstract = "T"
github_url = "https://github.com/shinshoji01"
# base_repo_dir = "/"
base_repo_dir = "/EDTTS-Demo/"
style_dir = base_repo_dir + "statics/"
fig_path = base_repo_dir + "images/inference.png"

            # <h5 class="mb-4">Sho Inoue<sup>1,*</sup>, Min-Jae Hwang<sup>1</sup></h5>
initial = f"""
<!DOCTYPE html>
<style>
    .red {{ color: red; font-weight: bold; }}
    .green {{ color: green; font-weight: bold; }}
    .blue {{ color: blue; font-weight: bold; }}
    .orange {{ color: orange; font-weight: bold; }}
    .grey {{ color: grey; font-weight: bold; }}
</style>
<html>
  <head>
    <meta charset="utf-8">
    <meta content="IE=edge" http-equiv="X-UA-Compatible">
    <meta content="width=device-width, initial-scale=1" name="viewport">
    <title>{webtitle}</title>
    <link href="{style_dir}bootstrap-5.2.3-dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="{style_dir}my.css" rel="stylesheet">
  </head>
  <body>
    <div class="container">
      <div class="row">
        <div class="container pt-5 mt-5 shadow p-5 mb-5 bg-white rounded">
          <div class="text-center">
            <h2>{title}</h2>
            <br>
            <h5 class="mb-4">TBD</h5>
          </div>
          <br>
          <figure class="text-center">
            <img src="{fig_path}" alt="overall diagram of the pipeline" class="img-fluid" style="width: 900px; height: auto;">
          </figure>
          <br>
          <h3>Abstract</h3>
          <p class="lead">
          {abstract}
          </p>
          <p class="lead">You can visit the project page of this paper: <a href="{github_url}">Github Repository</a>.
          </p>
          <p class="lead">This section presents audio samples produced by our proposed empathetic dialogue-generation system, Emotion-Aware DialogTTS (EDTTS). Each sample demonstrates how different models generate two-speaker conversational audio.
          </p>
        </div>
"""[1:]

closure = f"""
      </div>
    </div>
    <script src="{style_dir}jquery/jquery-3.7.1.slim.min.js"></script>
    <script src="{style_dir}bootstrap-5.2.3-dist/bootstrap.min.js"></script>
"""[1:]
closure += """
  </body>
  <script>
    $(function(){
        $("audio").on("play", function() {
            $("audio").not(this).each(function(index, audio) {
                audio.pause();
                audio.currentTime = 0;
            });
        });
    });
    </script>
</html>
"""[1:]

In [ ]:
experiments = {
    "syn": {
        "dialog/prompt": "Prompt",
        "dialog/textmspmanualvad---vad-linear-regression": "EmoDialogTTS (ours)",
        # "dialog/textmspmanualvad_synv3---vad-linear-regression": "EDTTS2 (ours)",
        # "dialog/textmspmanualvad_realfake_synv3---vad-linear-regression": "EDTTS3 (ours)",
        "dialog/baseline---prompt": "DialogTTS",
        "dialog/cosyvoice": "CosyVoice",
        "dialog/mooncast": "MoonCast + Speaker Separation",
        "dialog/mooncast_original": "MoonCast",
    },
    "real": {
        "dialog/prompt": "Prompt",
        "dialog/textmspmanualvad---vad-linear-regression": "EmoDialogTTS (ours)",
        # "dialog/textmspmanualvad_synv3---vad-linear-regression": "EDTTS2 (ours)",
        # "dialog/textmspmanualvad_realfake_synv3---vad-linear-regression": "EDTTS3 (ours)",
        "dialog/baseline---prompt": "DialogTTS",
        "dialog/cosyvoice": "CosyVoice",
        "dialog/mooncast": "MoonCast + Speaker Separation",
        "dialog/mooncast_original": "MoonCast",
    },
    "realprompt": {
        "dialog/prompt": "Prompt",
        "dialog/groundtruth": "Ground-Truth",
        "dialog/textmspmanualvad---vad-linear-regression": "EmoDialogTTS (ours)",
        # "dialog/textmspmanualvad_synv3---vad-linear-regression": "EDTTS2 (ours)",
        # "dialog/textmspmanualvad_realfake_synv3---vad-linear-regression": "EDTTS3 (ours)",
        "dialog/baseline---prompt": "DialogTTS",
        "dialog/cosyvoice": "CosyVoice",
        "dialog/mooncast": "MoonCast + Speaker Separation",
        "dialog/mooncast_original": "MoonCast",
    },
}

vad_base_model = "intensity/textvad---vad-linear-regression"
# vad_base_model = "intensity/textvad---prompt"
# vad_base_model = "intensity/textvad_realfake_synv3---vad-linear-regression"
intensity_list = ["0.0", "0.2", "0.4", "0.6", "0.8", "1.0"]
for emotion in target_emotions:
    experiments[f"intensity_{emotion}"] = {"intensity/prompt": "Prompt"}
    for intkey in intensity_list:
        experiments[f"intensity_{emotion}"][vad_base_model+f"___{intkey}"] = f"{emotion[0].upper()+emotion[1:]} Intensity {intkey}"

titles = {
    "syn": "Empathetic dialog generation (empathetic text dialogs, \"SynPrompts\")",
    "real": "Empathetic dialog generation (empathetic text dialogs, \"RealPrompts\")",
    "realprompt": "Speech dialog generation (ground-truth text dialogs, \"RealPrompts\")",
}
for emo in [emo[0].upper()+emo[1:] for emo in target_emotions]:
    titles["intensity_"+emo.lower()] = f"Emotion intensity controllability via valence-arousal-dominance controls (<span class=\"{emo2color[emo]}\">{emo}</span>)"

explanations = {
    "syn": "\
We constructed the empathetic text dialogues using a large language model (LLM). For the speech prompts, we concatenated utterance-level speech samples to simulate two-channel speech dialogues. \
<ul> \
  <li><b>Prompt</b>: The input speech prompts used for inference. To create two-speaker conversations, we combined single-speaker audio clips from the RAVDESS and ESD datasets. <br></li> \
  <li><b>EmoDialogTTS (ours)</b>: Our emotion-aware version of DialogTTS, designed to generate empathetic speech dialogues.<br></li> \
  <li><b>DTTS</b>: The original DialogTTS model, which does not use any emotion information when generating speech.<br></li> \
  <li><b>CosyVoice</b>: Speech generated by CosyVoice. Since CosyVoice produces single-utterance samples, we simply concatenated them to form dialogue samples. For the speech prompts, we used expressive speech samples to generate outputs with the target emotion labels.<br></li> \
  <li><b>MoonCast + Speaker Separation</b>: Speech dialogues generated with MoonCast. We then used a speaker-separation process to split the mixed audio into two separate speaker channels.<br></li> \
  <li><b>MoonCast</b>: Speech dialogues generated with MoonCast. <br></li> \
</ul> \
",
    "real": "\
We constructed the empathetic text dialogues using a large language model (LLM). For the speech prompts, we used real two-channel speech dialogues to more accurately simulate natural conversations. \
<ul> \
  <li><b>Prompt</b>: The input speech prompts used for inference. To create two-speaker conversations, we combined single-speaker audio clips from the RAVDESS and ESD datasets. <br></li> \
  <li><b>EmoDialogTTS (ours)</b>: Our emotion-aware version of DialogTTS, designed to generate empathetic speech dialogues.<br></li> \
  <li><b>DTTS</b>: The original DialogTTS model, which does not use any emotion information when generating speech.<br></li> \
  <li><b>MoonCast + Speaker Separation</b>: Speech dialogues generated with MoonCast. We then used a speaker-separation process to split the mixed audio into two separate speaker channels.<br></li> \
  <li><b>MoonCast</b>: Speech dialogues generated with MoonCast. <br></li> \
</ul> \
",
    "realprompt": "\
We used the ground-truth text dialogues and real two-channel speech prompts, used for direct comparison against the ground-truth. \
<ul> \
  <li><b>Prompt</b>: The input speech prompts used for inference. To create two-speaker conversations, we combined single-speaker audio clips from the RAVDESS and ESD datasets. <br></li> \
  <li><b>Ground-Truth</b>: The original, reference speech samples used as the standard for comparison.<br></li> \
  <li><b>EmoDialogTTS (ours)</b>: Our emotion-aware version of DialogTTS, designed to generate empathetic speech dialogues.<br></li> \
  <li><b>DTTS</b>: The original DialogTTS model, which does not use any emotion information when generating speech.<br></li> \
  <li><b>CosyVoice</b>: Speech generated by CosyVoice. Since CosyVoice produces single-utterance samples, we simply concatenated them to form dialogue samples. <br></li> \
  <li><b>MoonCast + Speaker Separation</b>: Speech dialogues generated with MoonCast. We then used a speaker-separation process to split the mixed audio into two separate speaker channels.<br></li> \
  <li><b>MoonCast</b>: Speech dialogues generated with MoonCast. <br></li> \
</ul> \
",
    "intensity_angry": "\
This section presents speech samples showing a range of emotion intensities. Each sample’s emotional intensity is quantified using valence-arousal-dominance (VAD) values. To characterize each emotion, we first computed an emotion vector by subtracting the VAD values of the neutral state from those of the target emotion. We then varied the magnitude of this vector in its respective direction to generate speech samples with different emotion intensities. \
",
    "intensity_happy": "\
This section presents speech samples showing a range of emotion intensities. Each sample’s emotional intensity is quantified using valence-arousal-dominance (VAD) values. To characterize each emotion, we first computed an emotion vector by subtracting the VAD values of the neutral state from those of the target emotion. We then varied the magnitude of this vector in its respective direction to generate speech samples with different emotion intensities. \
",
    "intensity_sad": "\
This section presents speech samples showing a range of emotion intensities. Each sample’s emotional intensity is quantified using valence-arousal-dominance (VAD) values. To characterize each emotion, we first computed an emotion vector by subtracting the VAD values of the neutral state from those of the target emotion. We then varied the magnitude of this vector in its respective direction to generate speech samples with different emotion intensities. \
",
}

In [ ]:
# files = glob.glob(base_dir+f"{base_model}/*/*/*/*.wav")
# filenames = list(set([p[len(base_dir)+len(base_model)+1:] for p in files]))
# filenames.sort()
# filenames = filenames[::2]
# np.random.seed(0)

In [ ]:
filenames_dir = {}
base_model = "dialog/baseline---prompt"
files = glob.glob(base_dir+f"{base_model}/syn/*/*/*/*.wav")
filenames = list(set([p[len(base_dir)+len(base_model)+1:] for p in files]))
filenames.sort()
filenames = [[fn.replace("/"+emo+"/", "/*/") for emo in emo2color if emo in fn][0] for fn in filenames[::2]]
np.random.seed(0)
filenames = [str(p) for p in list(np.random.choice(filenames, len(filenames), replace=False))]

newfiles = []
for f, fn in enumerate(filenames):
    np.random.seed(f)
    newfiles += [str(p)[len(base_dir)+len(base_model)+1:] for p in np.random.choice(glob.glob(base_dir+f"{base_model}/{fn}"), 2, replace=False)]
filenames_syn = [fn for fn in newfiles if not("davidaitest" in fn)]
filenames_real = [fn for fn in newfiles if "davidaitest" in fn]

filenames_dir["syn"] = filenames_syn
filenames_dir["real"] = filenames_real

files = glob.glob(base_dir+f"{base_model}/real/*/*/*/*.wav")
filenames = list(set([p[len(base_dir)+len(base_model)+1:] for p in files]))
filenames.sort()
np.random.seed(0)
filenames = [str(p) for p in np.random.choice(filenames, len(filenames))]

filenames_dir["realprompt"] = filenames

base_model = vad_base_model + f"___0.0"
files = glob.glob(base_dir+f"{base_model}/syn/*/*/*/*.wav")
files.sort()
files = [str(file) for file in np.random.choice(files, size=len(files), replace=False)]
filenames = [file[len(base_dir)+len(base_model)+1:] for file in files]
for emo in ["angry", "happy", "sad"]:
    filenames_dir[f"intensity_{emo}"] = [fn for fn in filenames if f"___{emo}.wav" in fn]

In [ ]:
extexts = {}
for exid in experiments:
    text = f"""
            <div class="container pt-5 mt-5 shadow p-5 mb-5 bg-white rounded">
              <h3>{titles[exid]}</h3>
              <p class="lead">
                    {explanations[exid]}
                    </p>
              <div class="table-responsive" style="overflow-x: scroll">
                <table class="table table-sm">
    """[1:]
    text += f"""
                  <thead>
                    <tr>
                      <th scope="col">ID</th>
    """[1:]
    for fn in filenames_dir[exid]:
        if "intensity" in exid:
            title = "/".join(fn.split("/")[1:3])
        else:
            title = "/".join(fn.split("/")[1:4])
        text += f"""
                      <th scope="col" class="text-center">{title}</th>
    """[1:]
    text += """
                    </tr>
                  </thead>
    """[1:]
    headerfn = text

    text = """
                  <tbody>
                    <tr>
                      <th scope="row" style="position: sticky; left: 0; z-index:10; opacity: 1.0; background-color: white;">Text</th>
    """[1:]
    for fn in filenames_dir[exid]:
        if "intensity" in exid:
            # emo = os.path.basename(fn).split("___")[-1:]
            # emo = emo[:-4]
            # html_table = f"Intensity: {intkey} ({emo})"
            transcript = load_json(base_dir + f"intensity/prompt/{fn}".replace(".wav", ".json"))["Turn 1"]["transcription"]
            html_table = transcript
        else:
            html_table = '<table style="border-collapse: separate; border-spacing: 0 5px;">'
            key = list(experiments[exid].keys())[0]
            dialog = load_json(base_dir + f"{key}/{fn}".replace(".wav", ".json"))
            for turnkey in dialog:
                turn = dialog[turnkey]
                emo = turn["emotion"]
                spk = turn["speaker"]
                spk = spk[0].upper() + spk[1:]

                html_table += "<tr>"
                transcript = text_normalization_dialogtts(turn["transcription"])
                if exid in ["realprompt"]:
                    spk = "Left" if spk=="User" else "Right"
                    html_table += (
                        f'<td style="text-align: center; vertical-align: middle; min-width: 100px;">'
                        f'<span style="font-size: 0.8rem;">'
                        f'<strong>{spk}</strong><br>'
                        f'</span>'
                        f'</td>'
                    )
                    html_table += f'<td style="text-align: center; vertical-align: middle; min-width: 300px;"><span style="font-size: 0.8rem;">{transcript}</span></td>'
                else:
                    emocolor = emo2color[emo]
                    html_table += (
                        f'<td style="text-align: center; vertical-align: middle; min-width: 100px;">'
                        f'<span style="font-size: 0.8rem;">'
                        f'<strong>{spk}</strong><br>'
                        f'(<span class="{emocolor}">{emo}</span>)'
                        f'</span>'
                        f'</td>'
                    )
                    html_table += f'<td style="text-align: center; vertical-align: middle; min-width: 300px;"><span style="font-size: 0.8rem;">{transcript}</span></td>'
                html_table += "</tr>"
            html_table += "</table>"
        text += f"""
                  <td>
                    <p>{html_table}</p>
                  </td>
    """[1:]
    text += """
                    </tr>
    """[1:]
    headertext = text

    text = ""
    for key in experiments[exid]:
        text += f"""
                    <tr>
                      <th scope="row" style="position: sticky; left: 0; z-index:10; opacity: 1.0; background-color: white;">{experiments[exid][key]}</th>
    """[1:]
        for fn in filenames_dir[exid]:
            wavfile = base_dir + f"{key}/{fn}"
            text += f"""
                      <td style="text-align: center; vertical-align: middle; min-width: 100px;">
    """[1:]
            text += f"""
                        <audio controls="" preload="none" style="width: 240px">
                          <source src="{wavfile}" type="audio/wav">
                        </audio>
    """[1:]
            text += f"""
                      </td>
    """[1:]
        text += """
                    </tr>
    """[1:]
    body = text

    text = """
                  </tbody>
                </table>
              </div>
              <p class="lead">* please scroll horizontally to explore additional columns in the table.</p>
            </div>
    """[1:]
    tableclosure = text
    extexts[exid] = headerfn + headertext + body + tableclosure
    extexts[exid] = "\n".join([a[4:] for a in extexts[exid].split("\n")])
    
wholetext = ""
wholetext += initial
for exid in experiments:
    wholetext += extexts[exid]
wholetext += closure
f = open("index.html", "w")
f.write(wholetext)
f.close()
# print(wholetext)